In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 3)

In [23]:
RAW_FILE = '../../data/raw/W251 ML Covid Image Analysis_March 5, 2022_23.06.csv'
OUT_FILE = '../../data/interim/cdetect/corrected_dataset.csv'
df = pd.read_csv(RAW_FILE, header=0, skiprows=[1,2])
df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,d_age_1,d_gender,d_vacc_status_1,d_race,d_race_6_TEXT,v_temp_unit,v_temp_c_1,v_temp_f_1,v_cough,s_antipyretic,s_odynophagia,s_odynophagia_days,s_dysphagia,s_dysphagia_days,t_rtpcr,t_rtpcr_date,t_ag,t_ag_date,t_med,d_dx,d_img1_Id,d_img1_Name,d_img1_Size,d_img1_Type,d_img2_Id,d_img2_Name,d_img2_Size,d_img2_Type,d_img3_Id,d_img3_Name,d_img3_Size,d_img3_Type,source
0,2022-02-06 13:57:32,2022-02-06 13:59:04,Survey Preview,NaN,100,91,True,2022-02-06 13:59:05,R_2b30KZiPb9AfAqB,NaN,NaN,NaN,NaN,47.587,-122.039,preview,EN,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_3kiHD0ZHiczR95f,WP_000215.jpg,5.667e+04,image/jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-10 18:11:11,2022-02-10 18:12:13,IP Address,172.226.6.21,100,62,True,2022-02-10 18:12:15,R_cHoU0OrlzVnbpIJ,NaN,NaN,NaN,NaN,34.054,-118.244,anonymous,EN,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_eG1AMXg7TDcp4NH,10E86F23-FEF0-4730-BF0B-047856305619.jpeg,2.014e+06,image/jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-02-11 20:33:40,2022-02-11 20:35:14,IP Address,98.203.232.40,100,93,True,2022-02-11 20:35:17,R_31pB9W7t53CMnvl,NaN,NaN,NaN,NaN,47.587,-122.039,anonymous,EN,50.0,Male,3.0,Asian,NaN,Fahrenheit,NaN,98.4,None,No,None,NaN,None,NaN,Not taken,NaN,Not taken,NaN,NaN,Normal,F_UPZDSsUUM05VdE5,20220211_193045.jpg,1.666e+06,image/jpeg,F_129EtL4sWo6Bn6T,20220211_193037.jpg,2.125e+06,image/jpeg,NaN,NaN,NaN,NaN,test
3,2022-02-11 20:41:55,2022-02-11 20:43:20,IP Address,98.203.232.40,100,84,True,2022-02-11 20:43:23,R_vpLYwUqW7mLyttn,NaN,NaN,NaN,NaN,47.587,-122.039,anonymous,EN,72.0,Female,3.0,Asian,NaN,Fahrenheit,NaN,98.4,None,No,None,NaN,None,NaN,Not taken,NaN,Not taken,NaN,NaN,Normal,F_2YzZN9NmVK7gTrH,20220211_192942.jpg,1.551e+06,image/jpeg,F_3Gyd8bY2uxrkuFj,20220211_192931.jpg,1.496e+06,image/jpeg,F_27PPpkHFKjkfqrQ,20220211_192931.jpg,1.496e+06,image/jpeg,test
4,2022-02-11 21:11:38,2022-02-11 21:14:00,IP Address,71.191.163.55,100,142,True,2022-02-11 21:14:03,R_1P7Ds55gFHWBsAe,NaN,NaN,NaN,NaN,38.565,-77.288,anonymous,EN,45.0,Male,3.0,Black or African American,NaN,Fahrenheit,NaN,98.7,None,No,None,NaN,None,NaN,Not taken,NaN,Not taken,NaN,NaN,Normal,F_2OI2GsyM6hCSN5V,IMG_8144.jpg,1.193e+05,image/jpeg,F_2TzaAZd8V2csgSV,IMG_8142.jpg,3.030e+05,image/jpeg,F_1ly25ImRHth7IUo,IMG_8141.jpg,6.112e+04,image/jpeg,test


Check the column names and find relevant fields only

In [3]:
df_cols = list(df.columns)
df_cols

['StartDate',
 'EndDate',
 'Status',
 'IPAddress',
 'Progress',
 'Duration (in seconds)',
 'Finished',
 'RecordedDate',
 'ResponseId',
 'RecipientLastName',
 'RecipientFirstName',
 'RecipientEmail',
 'ExternalReference',
 'LocationLatitude',
 'LocationLongitude',
 'DistributionChannel',
 'UserLanguage',
 'd_age_1',
 'd_gender',
 'd_vacc_status_1',
 'd_race',
 'd_race_6_TEXT',
 'v_temp_unit',
 'v_temp_c_1',
 'v_temp_f_1',
 'v_cough',
 's_antipyretic',
 's_odynophagia',
 's_odynophagia_days',
 's_dysphagia',
 's_dysphagia_days',
 't_rtpcr',
 't_rtpcr_date',
 't_ag',
 't_ag_date',
 't_med',
 'd_dx',
 'd_img1_Id',
 'd_img1_Name',
 'd_img1_Size',
 'd_img1_Type',
 'd_img2_Id',
 'd_img2_Name',
 'd_img2_Size',
 'd_img2_Type',
 'd_img3_Id',
 'd_img3_Name',
 'd_img3_Size',
 'd_img3_Type',
 'source']

We don't need most of the fields from Qualtrics except `ResponseId`. Also, pull the label column upfront for easy visualization

In [4]:

req_columns = ['d_dx','ResponseId']

d_cols = [c for c in df_cols if c[1] == '_' and c != 'd_dx'] 
req_columns.extend(d_cols)
ren_columns = [c if not c.endswith('_1') else c[:-2] for c in req_columns ]

In [5]:
df = df[req_columns]
print(f"DataFrame object now has {df.shape[0]} rows and {df.shape[1]} columns")
df.columns = ren_columns
df.describe()

DataFrame object now has 142 rows and 33 columns


,d_age,d_vacc_status,v_temp_c,v_temp_f,s_odynophagia_days,s_dysphagia_days,d_img1_Size,d_img2_Size,d_img3_Size
count,140.000,36.000,135.000,5.000,11.000,10.000,1.380e+02,3.000e+01,1.800e+01
mean,38.186,2.167,37.113,98.460,8.091,8.800,5.423e+05,1.413e+06,1.466e+06
std,19.034,0.941,0.756,0.134,4.847,5.007,1.529e+06,7.053e+05,8.545e+05
min,7.000,0.000,35.700,98.400,3.000,2.000,1.170e+04,1.398e+04,6.112e+04
25%,23.000,2.000,36.600,98.400,5.000,5.250,2.044e+04,9.810e+05,9.584e+05
50%,36.000,2.000,37.000,98.400,6.000,8.500,3.013e+04,1.379e+06,1.368e+06
75%,50.250,3.000,37.500,98.400,12.000,13.000,8.084e+05,1.650e+06,1.615e+06
max,91.000,3.000,40.200,98.700,16.000,16.000,1.619e+07,3.549e+06,3.571e+06


In [6]:
def eda(df,col_names=None, row_width=60 ):
    columns = list(df.columns) if col_names is None else col_names
    print('='*row_width)
    print('df')
    print(f"{len(columns)} Reported \t{df.shape[1]} Columns\t{df.shape[0]} Rows" )
    d_res = pd.DataFrame()
    for i,col in enumerate(columns):
        res = {}
        print('-'*row_width)
        dtype_name = df[col].dtype 
        res['dtype'] = dtype_name
        res['n'] = df[col].count()
        res['NaN'] = df[col].isnull().sum()
        res['unique'] = len(df[col].unique())
        print(f"  {col}")
        d_col = pd.DataFrame([res],index=[col])
        
        print(d_col.to_string(index=False))
        if res['unique'] < 15 or dtype_name == 'object':
            d_col_counts = df[col].value_counts(dropna=False)
            d_col_counts.name = 'Count'
            d_col_freq = df[col].value_counts(dropna=False, normalize=True)
            d_col_freq.name = 'Freq'
            df_col = pd.concat([d_col_counts,d_col_freq],axis=1)
            df_col = df_col.transpose()
            df_col.columns = [str(c)[:8] + ".." if len(str(c)) > 8 else str(c)[:8] for c in df_col.columns]
            if res['unique'] > 10:
                df_col_1 = df_col.iloc[:,:5]
                df_col_2 = df_col.iloc[:,-5:] 
                df_col = df_col_1.join(df_col_2, lsuffix='_1',rsuffix='_2')
            print(df_col.to_string())
            res['mode'] = df[col].dropna().mode()
        if dtype_name == "float":
            res['min'] = df[col].dropna().min()
            res['median'] = df[col].dropna().median()
            res['mode'] = df[col].dropna().mode()
            res['max'] = df[col].dropna().max()
        d_col = pd.DataFrame([res],index = [col])
        d_res = pd.concat([d_res,d_col])

    print('='*row_width)
    return d_res
            

In [7]:
eda(df)

df
33 Reported 	33 Columns	142 Rows
------------------------------------------------------------
  d_dx
  dtype    n  NaN  unique
 object  136    6       6
       Normal  Bacteria..   Covid  Viral    nan  Other
Count  53.000      49.000  24.000  7.000  6.000  3.000
Freq    0.373       0.345   0.169  0.049  0.042  0.021
------------------------------------------------------------
  ResponseId
  dtype    n  NaN  unique
 object  142    0     142
       R_2a9MEx..  R_3KIM2A..  R_3ek0NZ..  R_x431CM..  R_3PnPgE..  R_bvyd8M..  R_1joCAO..  R_OKnyQW..  R_3PzzY6..  R_psEraL..
Count       1.000       1.000       1.000       1.000       1.000       1.000       1.000       1.000       1.000       1.000
Freq        0.007       0.007       0.007       0.007       0.007       0.007       0.007       0.007       0.007       0.007
------------------------------------------------------------
  d_age
   dtype    n  NaN  unique
 float64  140    2      58
----------------------------------------------------

,dtype,n,NaN,unique,mode,min,median,max
d_dx,object,136,6,6,0 Normal dtype: object,NaN,NaN,NaN
ResponseId,object,142,0,142,0 R_0dE4arEYyn9VJOV 1 R_10vi04lh64FG...,NaN,NaN,NaN
d_age,float64,140,2,58,0 36.0 dtype: float64,7.0,3.600e+01,9.100e+01
d_gender,object,137,5,3,0 Male dtype: object,NaN,NaN,NaN
d_vacc_status,float64,36,106,5,0 3.0 dtype: float64,0.0,2.000e+00,3.000e+00
d_race,object,138,4,5,0 Other dtype: object,NaN,NaN,NaN
d_race_6_TEXT,object,8,134,3,0 Hispanic dtype: object,NaN,NaN,NaN
v_temp_unit,object,140,2,3,0 Celsius dtype: object,NaN,NaN,NaN
v_temp_c,float64,135,7,32,0 37.0 dtype: float64,35.7,3.700e+01,4.020e+01
v_temp_f,float64,5,137,3,0 98.4 dtype: float64,98.4,9.840e+01,9.870e+01


Based on the EDA we observe the following:
Data Structure:

- **Prescription** `d_dx` has 6 Non-Numeric values. Some of the test data was incorrectl included into the download and these values should be dropped.
- **Age, Gender, Race, Cough, Antipyertic (fever reducing medicines), Odynophagia (Painful Swallowing), Dysphagia (Difficulty Swallowing), RTPCR Test, Antigen (AG Test)** each have less than 6 records with invalid values. If all of these records overlap if `d_dx` value, then we can safely drop the values. 
- **Image 1 ID** has 4 fields without value, hence excluding test surveys all records appear to have at least one image. 
- There are 30 records with 2 images and only 18 with 3 images for the same patient.


Drop the records with NaN and check again.

In [17]:

dropNAColumns = ['d_age', 'd_dx']

df_clean = df.dropna(subset=dropNAColumns)
df_clean.reset_index(drop=True, inplace=True)
# Vaccination record is assumed to be non-vaccinated if user doesn't confirm
df_clean.loc[:,'d_vacc_status'] = df_clean['d_vacc_status'].fillna(0)
 

/home/rmadhira86/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Race, if Other has a text specified, then use that value.

In [18]:
df_clean.loc[(df_clean['d_race'] == "Other") & (df_clean['d_race_6_TEXT'].notna()), 'd_race'] = df_clean['d_race_6_TEXT']

Convert all the temperatures to Fahrenherit

In [19]:
df_clean['v_temperature'] = np.where(df_clean['v_temp_unit'] == 'Celsius', round(df_clean['v_temp_c']*1.8 +32 ,1), df_clean['v_temp_f'])

<ipython-input-19-0485d0150762>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['v_temperature'] = np.where(df_clean['v_temp_unit'] == 'Celsius', round(df_clean['v_temp_c']*1.8 +32 ,1), df_clean['v_temp_f'])


In [20]:
df_clean = df_clean.replace({
                 'd_dx':{"normal":0,"bacterial":1,"covid":2},
                 'd_gender':{'Male':-1,'Female':1},
                 's_antipyretic': {"No":0,"Yes":1},
                 's_odynophagia': {'None':0, 'Low':1, 'Medium':2,'High':3},
                 's_dysphagia' : {'None':0, 'Low':1, 'Medium':2,'High':3}})

In [21]:
df_clean['t_med2'] = [str.split(
    re.sub(r'^\s+|\s+$',  '',
        re.sub(r',[,\s]', ',',
               str(x).lower().replace(' and ',',').replace('antibiotics','antibiotic')
               )),
    ',')
  for x in df_clean['t_med']]
df_med = df_clean['t_med2'].explode().str.get_dummies().sum(level=0).add_prefix('t_med_')
df_clean = df_clean.join(df_med)
df_clean.head()

,d_dx,ResponseId,d_age,d_gender,d_vacc_status,d_race,d_race_6_TEXT,v_temp_unit,v_temp_c,v_temp_f,v_cough,s_antipyretic,s_odynophagia,s_odynophagia_days,s_dysphagia,s_dysphagia_days,t_rtpcr,t_rtpcr_date,t_ag,t_ag_date,t_med,d_img1_Id,d_img1_Name,d_img1_Size,d_img1_Type,d_img2_Id,d_img2_Name,d_img2_Size,d_img2_Type,d_img3_Id,d_img3_Name,d_img3_Size,d_img3_Type,v_temperature,t_med2,t_med_acetilcistein,t_med_acetoaminophen,t_med_advil,t_med_alprazolam,t_med_amlopine,t_med_antibiotic,t_med_anticoagulant,t_med_antifungal,t_med_antiinflamatory,t_med_antitusigen,t_med_atorvastantine,t_med_atorvastatine,t_med_baricicitinib,t_med_baricitinib,t_med_bariticinib,t_med_bisoproplol,t_med_cough drops,t_med_cough syrup,t_med_dexametasone,t_med_dexametosone,t_med_diclofenaco,t_med_duloxtine,t_med_enalapril,t_med_eritropoietin,t_med_guafenesina,t_med_heparin,t_med_heparine,t_med_iantiviral,t_med_insulin,t_med_insuline,t_med_losartan,t_med_nan,t_med_none,t_med_omeprazole,t_med_retroviral,t_med_telmisartan,t_med_troferit
0,Normal,R_31pB9W7t53CMnvl,50.0,0.0,3.0,Asian,NaN,Fahrenheit,NaN,98.4,None,0,0,NaN,0,NaN,Not taken,NaN,Not taken,NaN,NaN,F_UPZDSsUUM05VdE5,20220211_193045.jpg,1.666e+06,image/jpeg,F_129EtL4sWo6Bn6T,20220211_193037.jpg,2.125e+06,image/jpeg,NaN,NaN,NaN,NaN,98.4,[nan],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Normal,R_vpLYwUqW7mLyttn,72.0,1.0,3.0,Asian,NaN,Fahrenheit,NaN,98.4,None,0,0,NaN,0,NaN,Not taken,NaN,Not taken,NaN,NaN,F_2YzZN9NmVK7gTrH,20220211_192942.jpg,1.551e+06,image/jpeg,F_3Gyd8bY2uxrkuFj,20220211_192931.jpg,1.496e+06,image/jpeg,F_27PPpkHFKjkfqrQ,20220211_192931.jpg,1.496e+06,image/jpeg,98.4,[nan],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Normal,R_1P7Ds55gFHWBsAe,45.0,0.0,3.0,Black or African American,NaN,Fahrenheit,NaN,98.7,None,0,0,NaN,0,NaN,Not taken,NaN,Not taken,NaN,NaN,F_2OI2GsyM6hCSN5V,IMG_8144.jpg,1.193e+05,image/jpeg,F_2TzaAZd8V2csgSV,IMG_8142.jpg,3.030e+05,image/jpeg,F_1ly25ImRHth7IUo,IMG_8141.jpg,6.112e+04,image/jpeg,98.7,[nan],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Other,R_32SoCpPkA0chwa7,53.0,0.0,1.0,Other,NaN,Celsius,38.4,NaN,None,0,0,NaN,0,NaN,Negative,02-12-2022,Not taken,NaN,None,F_2tzLYf61Avt2Cht,7380641B-7A31-4EC6-B017-53928B96E2F2.jpeg,2.054e+06,image/jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.1,[none],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Normal,R_10vi04lh64FGSvp,23.0,0.0,3.0,White,NaN,Celsius,36.6,NaN,None,0,0,NaN,0,NaN,Negative,02-10-2022,Not taken,NaN,NaN,F_1dtz2Z4xWxQURGl,4AF218E5-8F2C-4865-AAA6-AD515C76F83D.jpeg,2.262e+06,image/jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.9,[nan],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [13]:
df_clean_cols = df_clean.columns
req_cols = ['d_dx','ResponseId','d_age','d_gender','d_vacc_status','d_race','v_temperature', 'v_cough']
symptom_cols = [c for c in df_clean_cols if c.startswith('s_') and not 'day' in c] 
treat_cols = [c for c in df_clean_cols if c.startswith('t_') and not 'date' in c and not 'med' in c] 
med_cols = [c for c in df_clean_cols if c.startswith('t_med_')]
final_cols = req_cols + symptom_cols + treat_cols + med_cols


In [16]:
df_final = df_clean[final_cols]
eda(df_final, req_cols+symptom_cols+treat_cols)

df
13 Reported 	50 Columns	135 Rows
------------------------------------------------------------
  d_dx
  dtype    n  NaN  unique
 object  135    0       5
       Normal  Bacteria..   Covid  Viral  Other
Count  52.000      49.000  24.000  7.000  3.000
Freq    0.385       0.363   0.178  0.052  0.022
------------------------------------------------------------
  ResponseId
  dtype    n  NaN  unique
 object  135    0     135
       R_2a9MEx..  R_3PnPgE..  R_1hyACv..  R_WwBuk2..  R_3lRoIs..  R_3m4zjQ..  R_231ylo..  R_bvyd8M..  R_1joCAO..  R_psEraL..
Count       1.000       1.000       1.000       1.000       1.000       1.000       1.000       1.000       1.000       1.000
Freq        0.007       0.007       0.007       0.007       0.007       0.007       0.007       0.007       0.007       0.007
------------------------------------------------------------
  d_age
   dtype    n  NaN  unique
 float64  135    0      54
------------------------------------------------------------
  d_gender
 

,dtype,n,NaN,unique,mode,min,median,max
d_dx,object,135,0,5,0 Normal dtype: object,NaN,NaN,NaN
ResponseId,object,135,0,135,0 R_0dE4arEYyn9VJOV 1 R_10vi04lh64FG...,NaN,NaN,NaN
d_age,float64,135,0,54,0 36.0 dtype: float64,13.0,36.0,91.0
d_gender,float64,134,1,3,0 0.0 1 1.0 dtype: float64,0.0,0.5,1.0
d_vacc_status,float64,135,0,4,0 0.0 dtype: float64,0.0,0.0,3.0
d_race,object,135,0,6,0 Other dtype: object,NaN,NaN,NaN
v_temperature,float64,135,0,32,0 98.6 dtype: float64,96.3,98.4,104.4
v_cough,object,135,0,3,0 None dtype: object,NaN,NaN,NaN
s_antipyretic,int64,135,0,2,0 0 dtype: int64,NaN,NaN,NaN
s_odynophagia,int64,135,0,4,0 0 dtype: int64,NaN,NaN,NaN


In [25]:
df_final.to_csv(OUT_FILE, index=False)